In [5]:
# Импортируем необходимые библиотеки:
# pandas для манипуляции данными,
# scikit-surprise для функциональности рекомендательных систем, такой как чтение датасетов и применение алгоритмов.
import pandas as pd
from surprise import Dataset, Reader, SlopeOne

# Загружаем датасет с рейтингами фильмов.
ratings = pd.read_csv("ratings.csv")

# Создаем таблицу сводки, чтобы увидеть рейтинги, которые пользователи ставят каждому фильму.
# Индекс: userId, Колонки: movieId, Значения: рейтинг.
user_movie_rating = ratings.pivot_table(index='userId', columns='movieId', values='rating')
# Показываем первые 100 строк для быстрого просмотра.
user_movie_rating.head(100)  

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Определяем объект Reader, указывая шкалу рейтинга.
reader = Reader(rating_scale=(0.5, 5))  # Рейтинги фильмов варьируются от 0.5 до 5.

# Загружаем данные из DataFrame.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Строим полный тренировочный набор данных.
trainset = data.build_full_trainset()

# Создаем экземпляр алгоритма SlopeOne.
algorithm = SlopeOne()
 # Обучаем модель на тренировочном наборе.
algorithm.fit(trainset) 

# Подготавливаем пустой список для тестового набора данных.
testset = []

# Проходим по всем пользователям и фильмам для создания тестового набора.
for uid in trainset.all_users():
    for iid in trainset.all_items():
        # Проверяем отсутствие пары (uid, iid) в тренировочном наборе.
        if not trainset.ur.get(uid) or (iid not in map(lambda x:x[0], trainset.ur[uid])):
            # Добавляем в тестовый набор с None в качестве фактического рейтинга.
            testset.append((trainset.to_raw_uid(uid), trainset.to_raw_iid(iid), None))

# Получаем предсказания для отсутствующих значений.
predictions = algorithm.test(testset)

# Подготавливаем список для хранения предсказанных рейтингов.
pred_ratings = []
for pred in predictions:
    pred_ratings.append([pred.uid, pred.iid, pred.est])

In [7]:
# Преобразуем список предсказаний в DataFrame.
pred_df = pd.DataFrame(pred_ratings, columns=['userId', 'movieId', 'rating'])

# Объединяем предсказанные рейтинги с исходными рейтингами.
complete_ratings = pd.concat([ratings[['userId', 'movieId', 'rating']], pred_df])

# Создаем итоговую таблицу сводки с полными данными, включающую как фактические, так и предсказанные рейтинги.
complete_user_movie_rating = complete_ratings.pivot_table(index='userId', columns='movieId', values='rating')

# Сортируем столбцы итоговой сводной таблицы согласно исходной таблице сводки рейтингов.
complete_user_movie_rating = complete_user_movie_rating[user_movie_rating.columns]

# Показываем первые 100 строк итоговой таблицы для быстрого просмотра.
complete_user_movie_rating.head(100)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.000000,4.070971,4.000000,2.712566,3.370275,4.000000,3.691134,3.610534,3.218810,4.266453,...,4.366379,4.366379,5.000000,5.000000,4.366379,5.000000,4.366379,4.366379,4.366379,4.066379
2,3.897793,3.241544,2.989686,1.769704,2.669013,4.071225,2.742431,2.927443,1.948276,3.768889,...,2.281609,2.281609,3.281609,3.281609,2.281609,3.281609,2.281609,2.281609,2.281609,3.448276
3,3.042642,2.256974,2.336431,0.916667,1.600888,2.716576,1.812206,1.801175,1.166958,2.558171,...,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897
4,3.623125,3.043395,3.046685,1.879241,2.157047,3.606155,2.590414,2.536638,2.083103,3.061995,...,2.555556,2.555556,3.555556,3.555556,2.555556,3.555556,2.555556,2.555556,2.555556,3.305556
5,4.000000,3.376510,3.269367,2.319918,2.769359,3.798697,3.079882,2.810812,2.564843,3.248291,...,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.236364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,5.000000,3.476035,3.282012,2.439004,2.750236,4.026857,3.042115,2.906452,2.694068,3.551693,...,3.884615,3.884615,3.884615,3.884615,3.884615,3.884615,3.884615,3.884615,3.884615,3.684615
97,4.610827,4.023266,3.856185,2.672222,3.405348,4.809946,3.628323,3.464052,3.255122,4.328936,...,4.194444,4.194444,5.000000,5.000000,4.194444,5.000000,4.194444,4.194444,4.194444,5.000000
98,4.500000,3.546909,3.189345,2.038331,2.868266,4.219464,2.700888,2.979873,2.326043,3.838066,...,2.759197,2.759197,3.759197,3.759197,2.759197,3.759197,2.759197,2.759197,2.759197,4.079710


In [8]:
# Экспортируем итоговую таблицу в CSV файл.
complete_user_movie_rating.to_csv("predicted_ratings.csv")